In [ ]:
import pandas as pd
import os

def concatenate_excel_csv_files_no_glob(folder_path):
    """
    Reads all Excel (.xlsx, .xls) and CSV (.csv) files from a specified folder 
    without using glob, and concatenates them into a single pandas DataFrame.

    Args:
        folder_path (str): The path to the directory containing the files.

    Returns:
        pd.DataFrame: A single DataFrame containing all the data.
    """
    # List to store individual DataFrames
    list_dfs = []

    # Iterate over all items in the directory

    length = len(os.listdir(folder_path))
    print(f"len is {length}")
    
    counter = 0
    for filename in os.listdir(folder_path):
        counter = counter + 1
        if counter % 100 == 0:
            print(f"{counter} /{length} {counter/length}") 
        # Construct full file path
        full_filepath = os.path.join(folder_path, filename)
        
        # Ensure it's a file and not a directory
        if os.path.isfile(full_filepath):
            if filename.endswith('.csv'):
                df = pd.read_csv(full_filepath)
                df['source_file'] = filename # Optional: add source file name
                
                # extracting out the symbol from the filenames
                c = "_" # character
                i= filename.index(c)
                res = filename[:i]
                df['symbol'] = res
                
                list_dfs.append(df)

    # Concatenate all DataFrames in the list
    if list_dfs:
        # ignore_index=True ensures a continuous index for the combined DataFrame
        combined_df = pd.concat(list_dfs, ignore_index=True) 
        return combined_df
    else:
        print("No files found to process.")
        return pd.DataFrame()

folder_directory = '.' 
merged_data = concatenate_excel_csv_files_no_glob(folder_directory)

print(merged_data.head())

In [ ]:
display(merged_data)

In [ ]:
print(merged_data.dtypes)

In [ ]:
merged_data['timestamp_dt'] = pd.to_datetime(merged_data['timestamp'])
print(merged_data.dtypes)

In [ ]:
df = merged_data.copy()

df['year'] = df['timestamp_dt'].dt.year
df['month'] = df['timestamp_dt'].dt.month
df['day'] = df['timestamp_dt'].dt.day
df['hour'] = df['timestamp_dt'].dt.hour
df['minute'] = df['timestamp_dt'].dt.minute
df['second'] = df['timestamp_dt'].dt.second

print(df.dtypes)

In [ ]:
display(df)

In [ ]:
symbol = "TSLA"
tick_type = "LAST"

df_subset = df.loc[(df["symbol"]==symbol) & (df["tick_type"] == tick_type)].copy()
display(df_subset)

In [ ]:
df_subset = df_subset.reset_index()

df_subset["counter"] = 1
df_subset["counter"] = df_subset["counter"].cumsum()
df_subset["hour_fraction"] = df_subset["hour"] + df_subset["minute"]/60 + df_subset["second"]/3600 + df_subset["counter"]/3600/60
display(df_subset)

In [ ]:

import pandas as pd

def aggregate_hourly_data(df, value_column):
    hourly_stats = df.groupby('agg_col')[value_column].agg([
        ('mean', 'mean'),
        ('std', 'std'),
        ('count', 'count'),
        ('max', 'max'),
        ('min', 'min'), 
        ('last', 'last')
    ]).reset_index()
    
    hourly_stats.rename(columns={'agg_cols': 'agg_col'}, inplace=True)
    
    return hourly_stats


In [ ]:

import pandas as pd

def create_date_string(df, col1='year', col2='month', col3='day', col4='hour', col5='minute', 
                       new_col_name='subset_category', separator='_'):
    
    # Create the concatenated column
    df[new_col_name] = (df[col1].astype(str) + separator + 
                        df[col2].astype(str) + separator + 
                        df[col3].astype(str) + separator + 
                        df[col4].astype(str) + separator + 
                        df[col5].astype(str))
    
    return df


In [ ]:
df_subset = create_date_string(df_subset)

In [ ]:
df_subset['agg_col'] = df_subset['source_file'].str[:19]

In [ ]:
display(df_subset)

In [ ]:
df_subset_agg = aggregate_hourly_data(df_subset, 'price')
display(df_subset_agg)

In [ ]:
df_subset_agg.to_csv("TSLA_test_data_agg_2025_12_16.csv")